In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv('Dataset2.csv', encoding='ISO-8859-1', delimiter=';')
#there was an issue with the loading of the dataset, hence the encoding = '' had to be used to convert

mapping_dict = {'Premier League': '1', 'La Liga': '2','Serie A': '3', 'Bundesliga': '4','Ligue 1' : '5'}
df['Comp'] = df['Comp'].map(mapping_dict)

print(df)



        Rk             Player Nation   Pos          Squad Comp   Age  Born  \
0        1         Max Aarons    ENG    DF   Norwich City    1  22.0  2000   
1        2   Yunis Abdelhamid    MAR    DF          Reims    5  34.0  1987   
2        3  Salis Abdul Samed    GHA    MF  Clermont Foot    5  22.0  2000   
3        4    Laurent Abergel    FRA    MF        Lorient    5  29.0  1993   
4        5        Charles Abi    FRA    FW  Saint-Étienne    5  22.0  2000   
...    ...                ...    ...   ...            ...  ...   ...   ...   
2916  2917   Martín Zubimendi    ESP    MF  Real Sociedad    2  23.0  1999   
2917  2918   Szymon ?urkowski    POL    MF         Empoli    3  24.0  1997   
2918  2919    Martin Ødegaard    NOR    MF        Arsenal    1  23.0  1998   
2919  2920        Milan ?uri?    BIH    FW    Salernitana    3  32.0  1990   
2920  2921      Filip ?uri?i?    SRB  MFFW       Sassuolo    3  30.0  1992   

      MP  Starts  ...   Off   Crs  TklW  PKwon  PKcon    OG  Re

In [16]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Load CSV into a pandas DataFrame
df = pd.read_csv('Dataset2.csv', encoding='ISO-8859-1', delimiter=';')

# Select relevant attributes
selected_attributes = ['Player', 'Comp', 'Pos', 'Goals', 'Shots', 'PasTotCmp', 'Assists', 'PasCrs', 'SCA', 'ScaDrib', 'GCA', 'TklWon', 'TklDriPast', 'PresSucc', 'Int', 'TouDefPen', 'Carries', 'CrdY', 'Recov', 'AerWon%']
selected_data = df[selected_attributes]

# Mapping categorical variables to numerical values
mapping_dict_comp = {'Premier League': 1, 'La Liga': 2, 'Serie A': 3, 'Bundesliga': 4, 'Ligue 1': 5}
selected_data.loc[:, 'Comp'] = selected_data['Comp'].map(mapping_dict_comp)

mapping_dict_pos = {'DF': 1, 'MF': 2, 'FW': 3, 'MFFW': 4, 'FWMF': 5, 'GK': 6, 'DFMF': 7, 'MFDF': 8, 'FWDF': 9, 'DFFW': 10}
selected_data.loc[:, 'Pos'] = selected_data['Pos'].map(mapping_dict_pos)

# Remove missing values
selected_data = selected_data.dropna()

# Extract player names for later use
player_names = selected_data['Player']
selected_data = selected_data.drop(columns=['Player'])

# Feature Scaling (Min-Max Scaling)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(selected_data)

# Calculate similarity matrix (Cosine Similarity)
similarity_matrix = cosine_similarity(scaled_data)

def find_similar_players(player_index, top_n=5):
    # Find the index of the player
    similar_players_indices = similarity_matrix[player_index].argsort()[-top_n - 1: -1][::-1]
    similar_players = [(player_names[i], similarity_matrix[player_index][i]) for i in similar_players_indices]
    return similar_players

# Example: Find similar players for a given player (replace 'player_name' with the name of the player)
player_name_to_find = 'Ferland Mendy'  # Replace 'player_name' with the actual player's name
if player_name_to_find in player_names.values:
    player_index = player_names[player_names == player_name_to_find].index[0]
    similar_players = find_similar_players(player_index)
    # Output similar players and their similarity scores
    print(f"Similar players for {player_name_to_find}:")
    for player, similarity in similar_players:
        print(f"{player}: Similarity Score - {similarity:.4f}")
else:
    print(f"{player_name_to_find} not found in the dataset.")



Similar players for Ferland Mendy:
Dani Alves: Similarity Score - 0.9866
Jordi Alba: Similarity Score - 0.9842
Luka Modri?: Similarity Score - 0.9842
Lucas Vázquez: Similarity Score - 0.9834
Toni Kroos: Similarity Score - 0.9786


In [26]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Load CSV into a pandas DataFrame
df = pd.read_csv('Dataset2.csv', encoding='ISO-8859-1', delimiter=';')

# Select relevant attributes
selected_attributes = ['Player', 'Comp', 'Pos', 'Goals', 'Shots', 'PasTotCmp', 'Assists', 'PasCrs', 'SCA', 'ScaDrib', 'GCA', 'TklWon', 'TklDriPast', 'PresSucc', 'Int', 'TouDefPen', 'Carries', 'CrdY', 'Recov', 'AerWon%']
selected_data = df[selected_attributes].copy()  # Ensure a copy is made

# Mapping categorical variables to numerical values
mapping_dict_comp = {'Premier League': 1, 'La Liga': 2, 'Serie A': 3, 'Bundesliga': 4, 'Ligue 1': 5}
selected_data.loc[:, 'Comp'] = selected_data['Comp'].map(mapping_dict_comp)

mapping_dict_pos = {'DF': 1, 'MF': 2, 'FW': 3, 'MFFW': 4, 'FWMF': 5, 'GK': 6, 'DFMF': 7, 'MFDF': 8, 'FWDF': 9, 'DFFW': 10}
selected_data.loc[:, 'Pos'] = selected_data['Pos'].map(mapping_dict_pos)

# Remove missing values
selected_data.dropna(inplace=True)  # Drop rows with missing values

# Extract player names for later use
player_names = selected_data['Player']
selected_data.drop(columns=['Player'], inplace=True)  # Drop player names column

# Feature Scaling (Standard Scaling)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(selected_data)

# Initialize KNN model
knn = NearestNeighbors(n_neighbors=2, algorithm='auto')
knn.fit(scaled_data)

def find_similar_player(player_name):
    if player_name in player_names.values:
        player_index = player_names[player_names == player_name].index[0]
        player_data = scaled_data[player_index].reshape(1, -1)
        distances, indices = knn.kneighbors(player_data)
        closest_player_index = indices.flatten()[1]  # Select the closest player (excluding the player itself)
        closest_player = player_names[closest_player_index]
        distance_to_player = distances.flatten()[1]
        return closest_player, distance_to_player
    else:
        return None, None

# Example: Find the closest related player for a given player (replace 'player_name' with the name of the player)
player_name_to_find = 'Dani Alves'  # Replace 'player_name' with the actual player's name
closest_player, distance_to_player = find_similar_player(player_name_to_find)
if closest_player and distance_to_player:
    print(f"Closest player to {player_name_to_find}: {closest_player}")
else:
    print(f"{player_name_to_find} not found in the dataset or has insufficient data.")


Closest player to Dani Alves: Jordi Alba


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


